In [ ]:
!pip install geffnet
!pip install timm
!pip install ttach

In [ ]:
class CFG:
    debug=False
    n_classes = 2
    lr=1e-4
    batch_size=16
    epochs=10# you can train more epochs
    seed=777
    n_fold=4
    warmup=-1
    device=1
    amp = True
    amp_inf = False
    smooth = False
    smooth_alpha = 0.1
    efnet_num = 10##:0,1,2
    drop_rate = 0.25
    crop = True##bool
    psuedo_label = False
    pseudo_predict = "2020-11-06_14:50:43.385109_predict.csv"
    TTA=True
    Attention=True
    white = False#ごましおになる
    model_name = "tf_efficientnet_b0_ns"
    zoom = True
SIZE = 512

In [ ]:
import pandas as pd 
import glob

open_nega = glob.glob("../input/glaucomadataset/Non Glaucoma/*png")
open_pos = glob.glob("../input/glaucomadataset/Glaucoma/*tif")



open_p = pd.DataFrame()
open_p["file"] = open_pos
open_p["label"] = 1

open_n = pd.DataFrame()
open_n["file"] = open_nega
open_n["label"] = 1

open_ = pd.concat([open_n,open_p])
print(open_.head())

open_["from_china"] = 0



china = pd.read_csv("../input/panda-efnetb2-180-weight/china_gla.csv")
china["file"] = ["../input/ocular-disease-recognition-odir5k/ODIR-5K/ODIR-5K/Training Images/{}".format(china["filename"].values[i]) for i in range(len(china))]
china["label"] = china["Gla"]
china_ = china.drop(["Unnamed: 0","Patient Age","ID","Patient Sex"],axis=1)
china_1 = china_.head(300)
china_0 = china_.tail(300)
cat_df = pd.concat([china_1,china_0])
cat_df = cat_df.reset_index(drop=True)

cat_df["from_china"] = 1

df = pd.concat([cat_df,open_])
df.to_csv("concat_df.csv",index=False)

In [ ]:
import geffnet
import sys
import gc
import os
import random
import time
from contextlib import contextmanager
from pathlib import Path
from collections import defaultdict, Counter

#from  torch.cuda.amp import autocast, GradScaler 
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import scipy as sp
import pydicom
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from functools import partial
from tqdm import tqdm
import ttach as tta##TTA
import timm

import matplotlib as mpl
mpl.use ( 'Agg') # must be written both in import intermediate
import matplotlib.pyplot as plt
import sklearn.metrics as metric
import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
import torchvision.models as models
from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip,RandomGamma, RandomRotate90,GaussNoise,Cutout
from albumentations.pytorch import ToTensorV2


# Utils
# ====================================================

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=42)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.current_device())

In [ ]:
class efenet_Model_attention(nn.Module):

    def __init__(self):
        super().__init__()
        if CFG.efnet_num ==0:
            m = geffnet.efficientnet_b0(pretrained=True, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==1:
            m = geffnet.efficientnet_b1(pretrained=True, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==2:
            m = geffnet.efficientnet_b2(pretrained=True, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==10:
            m = timm.create_model(CFG.model_name, pretrained=True, num_classes=CFG.n_classes)
        self.enc = nn.Sequential(*list(m.children())[:-3]) 
        nc = list(m.children())[-1].in_features
        self.atten = nn.Sequential(
                nn.Conv2d(nc, 1, 1),
                nn.Sigmoid())
        self.head = nn.Sequential(nn.AdaptiveAvgPool2d(1),nn.Flatten(),nn.Linear(nc,512),
                            nn.ReLU(),nn.BatchNorm1d(512), nn.Dropout(0.5),nn.Linear(512,2))
    def forward(self, x):
        x = self.enc(x)#ベースのモデルの流れに同じ
        atten = self.atten(x)
        #print(x.size(),atten.size())
        y = x*atten
        x = self.head(y)
        return x,atten

class baseline_model(nn.Module):

    def __init__(self):
        super().__init__()
        if CFG.efnet_num ==0:
            self.model = geffnet.efficientnet_b0(pretrained=True, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==1:
            self.model = geffnet.efficientnet_b1(pretrained=True, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==2:
            self.model = geffnet.efficientnet_b2(pretrained=True, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==10:
            self.model = timm.create_model(CFG.model_name, pretrained=False, num_classes=CFG.n_classes)
        elif CFG.efnet_num<3:
            self.model.classifier = nn.Linear(self.model.classifier.in_features, 2)

    def forward(self, x):
        
        if CFG.efnet_num ==10:
            x = self.model(x)
        else:
            x = self.model(x)#ベースのモデルの流れに同じ
        return x

class baseline_Model_attention(nn.Module):

    def __init__(self):
        super().__init__()
        if CFG.efnet_num ==0:
            m = geffnet.efficientnet_b0(pretrained=False, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==1:
            m = geffnet.efficientnet_b1(pretrained=False, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==2:
            m = geffnet.efficientnet_b2(pretrained=False, drop_rate=CFG.drop_rate)
        elif CFG.efnet_num ==10:
            m = timm.create_model(CFG.model_name, pretrained=True, num_classes=CFG.n_classes)
        self.enc = nn.Sequential(*list(m.children())[:-3]) 
        nc = list(m.children())[-1].in_features
        self.atten = nn.Sequential(
                nn.Conv2d(nc, 1, 1),
                nn.Sigmoid())
        self.head = nn.Sequential(nn.AdaptiveAvgPool2d(1),nn.Flatten(),nn.Linear(nc,512),
                            nn.ReLU(),nn.BatchNorm1d(512), nn.Dropout(0.5),nn.Linear(512,2))
    def forward(self, x):
        x = self.enc(x)#ベースのモデルの流れに同じ
        atten = self.atten(x)
        #print(x.size(),atten.size())
        y = x*atten
        x = self.head(y)
        return x,atten


In [ ]:
import math
def get_pad_width(im, new_shape, is_rgb=True):
    pad_diff = new_shape - im.shape[0], new_shape - im.shape[1]
    t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
    l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
    if is_rgb:
        pad_width = ((t,b), (l,r), (0, 0))
    else:
        pad_width = ((t,b), (l,r))
    return pad_width

def crop_object(img, thresh=10, maxval=200, square=False):
    """
    Source: https://stackoverflow.com/questions/49577973/how-to-crop-the-biggest-object-in-image-with-python-opencv
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)# convert to grayscale
    #plt.imshow(gray,cmap="gray")
    #plt.show()#普通に白黒のがみえる
    # threshold to get just the signature (INVERTED)
    retval, thresh_gray = cv2.threshold(gray, thresh=thresh, maxval=maxval, type=cv2.THRESH_BINARY)
    #plt.imshow(thresh_gray,cmap="gray")
    #plt.show()
    contours, hierarchy = cv2.findContours(thresh_gray,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    #https://qiita.com/anyamaru/items/fd3d894966a98098376c
    # Find object with the biggest bounding box
    mx = (0,0,0,0)      # biggest bounding box so far
    mx_area = 0
    for cont in contours:
        x,y,w,h = cv2.boundingRect(cont)
        area = w*h
        if area > mx_area:
            mx = x,y,w,h
            mx_area = area
    x,y,w,h = mx#(0,0,0,0)なのはcontoursに何も入ってないから
    crop = img[y:y+h, x:x+w]
    if square:
        pad_width = get_pad_width(crop, max(crop.shape))
        crop = np.pad(crop, pad_width=pad_width, mode='constant', constant_values=255)
    if CFG.white:
        black = [0, 0, 0]
        white = [255, 255, 255]
        crop[np.where((crop == black).all(axis=2))] = white
    if CFG.zoom:
        h = crop.shape[0]
        w = crop.shape[1]
        h_ = int(h*0.15)
        w_ = int(w*0.15)
        crop = crop[h_:h-h_,w_:w-w_]
    return crop


In [ ]:
##train_test_split
from sklearn.model_selection import train_test_split
train, test = train_test_split(cat_df, test_size=0.3,stratify = cat_df["label"], random_state=2020)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
##train_valid_split
if CFG.debug:
    folds = train.sample(n=200, random_state=CFG.seed).reset_index(drop=True).copy()
else:
    folds = train.copy()
train_labels = folds["label"].values
kf = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for fold, (train_index, val_index) in enumerate(kf.split(folds.values, train_labels)):
    print("num_train,val",len(train_index),len(val_index),len(val_index)+len(train_index))
    folds.loc[val_index, 'fold'] = int(fold)

folds['fold'] = folds['fold'].astype(int)
folds.to_csv('folds.csv', index=None)
folds.head()

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df,transform1=None, transform2=None):
        self.df = df
        self.transform = transform1
        self.transform_ = transform2
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df['file'].values[idx]
        image = cv2.imread(file_path)
        if CFG.crop:
            image = crop_object(image)
        try:
            image = cv2.resize(image,(SIZE,SIZE))
        except Exception as e:
            print(file_path)
        label_ = self.df["label"].values[idx]
        if self.transform:
            image = self.transform(image=image)['image']
        if self.transform_:
            image = self.transform_(image=image)['image']

        
            
        label = torch.tensor(label_)
        #print(label_,type(label_),label,label.size())
        
        return image, label

In [ ]:
import sklearn.metrics as metric

def auc(y,y_hat):
    return metric.roc_auc_score(y,y_hat)


In [ ]:
def get_transforms1(*, data):

    #train,valid以外だったら怒る
    
    if data == 'train':
        return Compose([
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            #GaussNoise(p=0.5),
            #RandomRotate90(p=0.5),
            #RandomGamma(p=0.5),
            #RandomAugMix(severity=3, width=3, alpha=1., p=0.5),
            #GaussianBlur(p=0.5),
            #GridMask(num_grid=3, p=0.3),
            #Cutout(p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        ])
    
    elif data == 'valid':
        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            )
        ])

def to_tensor(*args):

        return Compose([
            ToTensorV2()
        ])

glaucomadataset由来のデータは学習済みモデルに通したベクトルを次元圧縮すると割とはっきり疾患か否かが別れていることがわかる。

OScularのデータセット（以下中国産のデータセットと呼ぶ）は分離が難しそう。

In [ ]:
!pip install --upgrade pip
!pip install -U torch

In [ ]:
@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')

    
def init_logger(log_file='train.log'):
    from logging import getLogger, DEBUG, FileHandler,  Formatter,  StreamHandler
    
    log_format = '%(asctime)s %(levelname)s %(message)s'
    
    stream_handler = StreamHandler()
    stream_handler.setLevel(DEBUG)
    stream_handler.setFormatter(Formatter(log_format))
    
    file_handler = FileHandler(log_file)
    file_handler.setFormatter(Formatter(log_format))
    
    logger = getLogger('RSNA2020')
    logger.setLevel(DEBUG)
    logger.addHandler(stream_handler)
    logger.addHandler(file_handler)
    
    return logger
import datetime
dt_now = datetime.datetime.now()
dt_now_ = str(dt_now).replace(" ","_")
print("実験開始",dt_now_)
LOG_FILE = 'train{}.log'.format(dt_now_)
LOGGER = init_logger(LOG_FILE)

In [ ]:
def onehot_encoding(label, n_classes):
    """
  　torch,tensor(k_0,k_1,k_2,)k_iは0~n_classes-1のどれか→one_hot
  　"""
    return torch.eye(n_classes)[label.type(torch.long)]
def label_smoothing(label,epsilon,n_classes):
    onehot = onehot_encoding(label, n_classes).float()
    label = onehot * (1 - epsilon) + torch.ones_like(onehot)* epsilon/n_classes
    return label

def cross_entropy_loss(input, target, reduction):
    logp = nn.functional.log_softmax(input, dim=1)
    loss = torch.sum(-logp * target, dim=1)
    if reduction == 'none':
        return loss
    elif reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    else:
        raise ValueError(
            '`reduction` must be one of \'none\', \'mean\', or \'sum\'.')


def train_fn(fold):
    print(f"### fold: {fold} ###")

        
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    train_dataset = TrainDataset(folds.loc[trn_idx].reset_index(drop=True), 
                                 transform1=get_transforms1(data='train'),transform2=to_tensor())#
    valid_dataset = TrainDataset(folds.loc[val_idx].reset_index(drop=True), 
                                 transform1=get_transforms1(data='valid'),transform2=to_tensor())#

    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)
    
    if CFG.Attention:
        model = efenet_Model_attention()
    else:
        model = efnet_model()
    model.to(device)
    
    optimizer = Adam(model.parameters(), lr=CFG.lr, amsgrad=False)
    #scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, eps=1e-6)
    #scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=0.001)
    
    criterion = nn.CrossEntropyLoss()#weight = class_weight
    softmax = nn.Softmax(dim = 1)
    scaler = torch.cuda.amp.GradScaler()
    #criterion = FocalLoss_BCE()#doesn't work
    best_score = 0
    best_loss = np.inf
    best_preds = None
    
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()

        model.train()

        if CFG.warmup>epoch:
            print("freeze")
            for param in model.model.parameters():
                param.requires_grad = False
            for param in model.model.classifier.parameters():
                param.requires_grad = True
        
        if CFG.warmup<=epoch:
            print("unfreeze")
            for param in model.parameters():
                param.requires_grad = True
        avg_loss = 0.

        tk0 = tqdm(enumerate(train_loader), total=len(train_loader))

        for i, (images, labels) in tk0:
            optimizer.zero_grad()

            images = images.to(device)
            smooth_labels = label_smoothing(labels,CFG.smooth_alpha,2).to(device)
            #print("smoooth",smooth_labels.size())#torch.Size([64, 3, 512, 512, 3])
            labels = labels.to(device)
            if CFG.Attention:
                y_preds,attention = model(images.float())
            else:
                y_preds = model(images.float())
            if CFG.smooth:
                loss = cross_entropy_loss(y_preds, smooth_labels,"mean")
            else:
                loss = criterion(y_preds, labels.long())
            loss.backward()
            optimizer.step()
                #optimizer.zero_grad()

            avg_loss += loss.item() / len(train_loader)
        model.eval()
        avg_val_loss = 0.
        preds = []
        valid_labels = []
        masks = []
        tk1 = tqdm(enumerate(valid_loader), total=len(valid_loader))

        for i, (images, labels) in tk1:

            images = images.to(device)
            smooth_labels = label_smoothing(labels,CFG.smooth_alpha,2).to(device)
            labels = labels.to(device)

            with torch.no_grad():
                if CFG.Attention:
                    y_preds,attention = model(images.float())
                    masks.append(attention.to("cpu").detach().numpy())
                else:
                    y_preds = model(images.float())
                if CFG.smooth:
                    loss = cross_entropy_loss(y_preds, smooth_labels,"mean")
                else:
                    loss = criterion(y_preds,labels.long())



            valid_labels.append(labels.to('cpu').numpy())
            softmax = nn.Softmax(dim = 1)
            y_preds = softmax(y_preds)
            #if i ==0:
                #print(y_preds.shape,y_preds.dtype,y_preds)#torch.Size([16, 5]) torch.float32

            preds.append(y_preds.to('cpu').numpy())
            avg_val_loss += loss.item() / len(valid_loader)
            
        #scheduler.step(avg_val_loss)
            
        preds = np.concatenate(preds)
        if CFG.Attention:
            masks = np.concatenate(masks)
        #print(preds.shape)
        valid_labels = np.concatenate(valid_labels)
        #valid_labels = np.identity(5)[valid_labels]

        print(preds.shape,valid_labels.shape)

        score = auc(valid_labels,preds[:,1])

        elapsed = time.time() - start_time
        LOGGER.debug(f'  Epoch {epoch+1} - avg_train_loss: {avg_loss:.6f}  avg_val_loss: {avg_val_loss:.6f}  time: {elapsed:.0f}s')
        
        if score>best_score:#loglossのスコアが良かったら予測値を更新...best_epochをきめるため
            best_score = score
            best_preds = preds
            LOGGER.debug(f'  Epoch {epoch+1} - Save Best Score: {best_score:.4f}')
            torch.save(model.state_dict(), f'fold{fold}_{dt_now_}_baseline.pth')#各epochのモデルを保存。。。best_epoch終了時のモデルを推論に使用する？
    
    return best_preds, valid_labels,masks


In [ ]:

preds = []
valid_labels = []
models =[]
for fold in range(CFG.n_fold):
    _preds, _valid_labels,_model = train_fn(fold)
    preds.append(_preds)
    valid_labels.append(_valid_labels)
    models.append(_model)
    
##
preds_ = np.concatenate(preds)
valid_labels_ = np.concatenate(valid_labels)

score = auc(valid_labels_,preds_)
import datetime

dt_now = datetime.datetime.now()
print("現在時刻",dt_now)
print("=====AUC(CV)======",score)
    

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df, transform1=None, transform2=None):
        self.df = df
        self.transform = transform1
        self.transform_ = transform2
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.df['file'].values[idx]
        image = cv2.imread(file_path)
        if CFG.crop:
            image = crop_object(image)
        image = cv2.resize(image,(SIZE,SIZE))
        label = self.df["label"].values[idx]
        if self.transform:
            image = self.transform(image=image)['image']
        if self.transform_:
            image = self.transform_(image=image)['image']

        
        return image


def inference(model, test_loader, device):

    transforms = tta.Compose([
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Rotate90(angles=[0, 180]),
        tta.Multiply(factors=[0.9, 1, 1.1]),        
    ])
    model.to(device) 
    if CFG.TTA and CFG.Attention:
        model = model
    elif CFG.TTA:
        model = tta.ClassificationTTAWrapper(model, transforms)
    
    model.to(device) 
    model.eval()
    probs = []
    masks = []

    for i, images in tqdm(enumerate(test_loader), total=len(test_loader)):
            
        images = images.to(device)
        with torch.no_grad():
            if CFG.Attention and CFG.Attention:
                attention_ = []
                for transformer in transforms:
                    augmented_image = transformer.augment_image(images.float())
                    y_preds,attention = model(augmented_image)
                    attention = transformer.deaugment_mask(attention)
                    attention_.append(attention)
                attention = torch.stack(attention_)
                attention = torch.mean(attention,axis=0)
            elif CFG.Attention:
                y_preds,attention = model(image.float())

            else:
                y_preds = model(images)
            softmax = nn.Softmax(dim = 1)
            y_preds = softmax(y_preds)
            
            
        probs.append(y_preds.to('cpu').numpy())
        if CFG.Attention:
            masks.append(attention.to("cpu").detach().numpy())

    probs = np.concatenate(probs)
    if CFG.Attention:
        masks = np.concatenate(masks)
        print(masks.shape)


    
    return probs,masks

In [ ]:
def submit():
        print('run inference')
        test_dataset = TestDataset(test, transform1=get_transforms1(data='valid'),transform2=to_tensor())
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)
        probs = []
        masks = []
        for fold in range(CFG.n_fold):
            weights_path = f"../input/panda-efnetb2-180-weight/fold{fold}_2020-11-24_022939.936386_baseline.pth"
            if CFG.Attention:
                model = baseline_Model_attention()
            else:
                model = baseline_model()
            state_dict = torch.load(weights_path,map_location=device)
            model.load_state_dict(state_dict)
            _probs,_mask = inference(model, test_loader, device)
            probs.append(_probs)
            masks.append(_mask)

        probs = np.mean(probs, axis=0)
        if CFG.Attention:
            masks = np.mean(masks, axis=0)

        return probs,masks

predict,masks= submit()
if CFG.Attention:
    np.save("attention_masks.npy",masks)
print(test.head())
print(test["label"].values.shape,predict.shape)
score = auc(test["label"].values,predict[:,1])
print("=====AUC(inner_test)======",score)
test["predict"] = predict[:,1]
test.to_csv("predict.csv",index = False)

可視化

In [ ]:
masks = np.load("attention_masks.npy")
%matplotlib  inline
for idx in tqdm(range(len(test))):
    file_path = test['file'].values[idx]
    label  = test['label'].values[idx]
    pred = test['predict'].values[idx]
    mask = masks[idx,:,:,:].transpose((1,2,0))[:,:,0]
    image = cv2.imread(file_path)
    image = crop_object(image)
    image = cv2.resize(image,(SIZE,SIZE))
    plt.title(f"{file_path}__label{label}_pred{pred}")
    plt.imshow(image)
    plt.show()
    plt.imshow(mask,cmap='jet')
    plt.show()
    
    
    